In [2]:
# pip install torch 

In [3]:
import pandas as pd
import altair as alt
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import timedelta
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

Salmon Detection Data

In [5]:
# salmon1 = pd.read_csv('data/data_salmon1.csv')
# salmon1['earliest_detect_date'] = pd.to_datetime(salmon1['earliest_detect_date'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce') # convert to date data type 
# salmon1['earliest_detect_date'] = pd.to_datetime(salmon1['earliest_detect_date']).dt.floor('d') # keep date but not time 
# salmon1.head()

In [6]:
# salmon_river = salmon1[salmon1['source'] == 'river' ]
# salmon_river.head() 

In [7]:
# locations = salmon_river['watershed'].unique()

# charts = []

# for loc in locations:
#     salmon_filtered = salmon_river[salmon_river['watershed'] == loc]
    
#     chart = alt.Chart(salmon_filtered).mark_bar().encode(
#         x=alt.X('earliest_detect_date:T', title='Date'), 
#         y=alt.Y('count()', title='Number of Records'),  
#         tooltip=[alt.Tooltip('earliest_detect_date:T', title='Date'), alt.Tooltip('count()', title='Records')]
#     ).properties(
#         title=f'Detections for {loc}'
#     )
    
#     charts.append(chart)

# for chart in charts:
#     chart.display()

EDA of the detection data. Since only Cowichan and Englishman have sufficient data, the model building will only consider these two watersheds. 

Flow Data

| Location    | Station Number |
|-------------|----------------|
| cowichan    | 08HA003        |
| englishman  | 08HA001        |

In [8]:
flow_ori = pd.read_csv('data/flow_2023.csv')
flow_columns = [col for col in flow_ori.columns if 'FLOW' in col]
flow_long = pd.melt(flow_ori, id_vars=["STATION_NUMBER", "YEAR", "MONTH"], value_vars=flow_columns, var_name="FlowType", value_name="FlowValue")
flow_long['FlowType'] = flow_long['FlowType'].str.replace("FLOW", "").astype(int)
sorted_flow = flow_long.sort_values(by=["STATION_NUMBER", "YEAR", "MONTH", "FlowType"])
sorted_flow.rename(columns={'FlowType': 'DAY'}, inplace=True)
sorted_flow = sorted_flow.dropna()
sorted_flow['Date'] = pd.to_datetime(sorted_flow[['YEAR', 'MONTH', 'DAY']])
sorted_flow = sorted_flow.reset_index(drop=True)
sorted_flow.head()

,STATION_NUMBER,YEAR,MONTH,DAY,FlowValue,Date
0,08HA001,2013,1,1,11.50,2013-01-01
1,08HA001,2013,1,2,10.30,2013-01-02
2,08HA001,2013,1,3,9.39,2013-01-03
3,08HA001,2013,1,4,8.91,2013-01-04
4,08HA001,2013,1,5,8.65,2013-01-05


In [9]:
flow_cow = sorted_flow[sorted_flow['STATION_NUMBER'] == '08HA003']
flow_cow.head() 

,STATION_NUMBER,YEAR,MONTH,DAY,FlowValue,Date
4018,08HA003,2013,1,1,10.40,2013-01-01
4019,08HA003,2013,1,2,9.18,2013-01-02
4020,08HA003,2013,1,3,8.10,2013-01-03
4021,08HA003,2013,1,4,7.69,2013-01-04
4022,08HA003,2013,1,5,7.48,2013-01-05


In [10]:
# df_filtered = sorted_flow[sorted_flow['STATION_NUMBER'] == "08HA003"]
# cowichan_flow_chart = alt.Chart(df_filtered).mark_line(point=False, color = 'red').encode(
#     x='Date:T',  
#     y='FlowValue:Q',  
#     tooltip=['YEAR', 'MONTH', 'FlowValue']  
# ).properties(
#     width=600,
#     height=300,
#     title='Flow of Cowichan'
# )

# cowichan_flow_chart 

Temperature Data 

In [11]:
tem_cow = pd.read_csv('data/northcochiwan_daily_temp-2.csv')
tem_cow.head() 
# tem_cow['UTC_DATE'] = pd.to_datetime(tem_cow['UTC_DATE']).dt.normalize()
# tem_cow_daily = tem_cow.groupby('UTC_DATE').mean().reset_index() 
# tem_cow_daily = tem_cow_daily.drop(['Unnamed: 0', 'WINDCHILL'], axis=1)
# tem_cow_daily.head() 

,UTC_DATE,RELATIVE_HUMIDITY,WIND_SPEED,TEMP,WINDCHILL,DEW_POINT_TEMP
0,2013-09-02,75.818182,2.727273,19.127273,NaN,14.372727
1,2013-09-03,83.125000,2.458333,18.045833,NaN,14.766667
2,2013-09-04,85.791667,2.000000,17.062500,NaN,14.379167
3,2013-09-05,94.708333,1.541667,16.837500,NaN,15.900000
4,2013-09-06,91.916667,1.583333,16.954167,NaN,15.504167


In [12]:
# chartcctemp = alt.Chart(tem_cow_daily).mark_line(point=False, color = 'yellow').encode(
#     x='UTC_DATE:T',  
#     y='TEMP:Q',  
#     tooltip=['UTC_DATE', 'TEMP', 'RELATIVE_HUMIDITY', 'WIND_SPEED', 'DEW_POINT_TEMP']  
# ).properties(
#     width=600,
#     height=300,
#     title='Water Temperature of Cochiwen'
# )
# chartcctemp

Cowichan df 

In [13]:
# tem_cow_daily.rename(columns={'UTC_DATE': 'Date'}, inplace=True)
# tem_cow_daily.tail() 

In [14]:
# flow_cow = sorted_flow[sorted_flow['STATION_NUMBER'] == '08HA003'][['Date', 'FlowValue']].reset_index(drop=True)
# flow_cow.tail() 

In [15]:
# salmon_river_co = salmon_river[salmon_river['species']== 'co']
# salmon_river_ck = salmon_river[salmon_river['species']== 'ck']

In [16]:
# salmon_cow1_long_co = salmon_river_co[salmon_river_co['watershed']== 'cowichan'][['earliest_detect_date', 'tagid']]
# salmon_cow1_co = salmon_cow1_long_co.groupby('earliest_detect_date').agg(count=('tagid', 'nunique')).reset_index()
# salmon_cow1_co.rename(columns={'earliest_detect_date': 'Date'}, inplace=True)
# salmon_cow1_co.head() 

In [17]:
# salmon_cow1_long_ck = salmon_river_ck[salmon_river_ck['watershed']== 'cowichan'][['earliest_detect_date', 'tagid']]
# salmon_cow1_ck = salmon_cow1_long_ck.groupby('earliest_detect_date').agg(count=('tagid', 'nunique')).reset_index()
# salmon_cow1_ck.rename(columns={'earliest_detect_date': 'Date'}, inplace=True)
# salmon_cow1_ck.head() 

In [18]:
# firstmerge = pd.merge(tem_cow_daily, flow_cow, on='Date', how='left')
# full_cow_co = pd.merge(firstmerge, salmon_cow1_co, on='Date', how='left')
# full_cow_ck = pd.merge(firstmerge, salmon_cow1_ck, on='Date', how='left')

In [19]:
# full_cow_co.head()

In [20]:
# full_cow_ck.head()

Create appropriate dataframe 

In [21]:
# # For simplicity, only using 2022, 2023 data for now 
# full_cow2223_co = full_cow_co[full_cow_co['Date'].dt.year.isin([2022, 2023])].reset_index()
# full_cow2223_co.drop('index', axis=1, inplace=True)
# full_cow2223_co['count'] = full_cow2223_co['count'].fillna(0)
# full_cow2223_co.rename({'TEMP': 'temperature', 'RELATIVE_HUMIDITY': 'relative_humidity', 'WIND_SPEED': 'wind_speed', 'DEW_POINT_TEMP': 'dew_point_temperature', 'FlowValue': 'flow_value', 'count': 'salmon_count'}, axis=1, inplace=True)

# full_cow2223_co.head() 

In [22]:
# # For simplicity, only using 2022, 2023 data for now 
# full_cow2223_ck = full_cow_ck[full_cow_ck['Date'].dt.year.isin([2022, 2023])].reset_index()
# full_cow2223_ck.drop('index', axis=1, inplace=True)
# full_cow2223_ck['count'] = full_cow2223_ck['count'].fillna(0)
# full_cow2223_ck.rename({'TEMP': 'temperature', 'RELATIVE_HUMIDITY': 'relative_humidity', 'WIND_SPEED': 'wind_speed', 'DEW_POINT_TEMP': 'dew_point_temperature', 'FlowValue': 'flow_value', 'count': 'salmon_count'}, axis=1, inplace=True)

# full_cow2223_ck.head() 

In [23]:
# # visualize all data 
# full_cow2223_long_co = full_cow2223_co.melt('Date', var_name='Measurement', value_name='Value')

# chart_full_cow2223_co = alt.Chart(full_cow2223_long_co).mark_line().encode(
#     x='Date:T',
#     y='Value:Q',
#     color='Measurement:N'
# ).properties(
#     width=600,
#     height=400
# )
# chart_full_cow2223_co

In [24]:
# full_cow2223_long_ck = full_cow2223_ck.melt('Date', var_name='Measurement', value_name='Value')

# chart_full_cow2223_ck = alt.Chart(full_cow2223_long_ck).mark_line().encode(
#     x='Date:T',
#     y='Value:Q',
#     color='Measurement:N'
# ).properties(
#     width=600,
#     height=400
# )
# chart_full_cow2223_ck 

Lag data 

In [25]:
# # creating lag data 
# full_cow2223_co.set_index('Date', inplace=True)

# def lag_df(df, lag=1):
#     lagged_df = pd.DataFrame()
#     for col in df.columns:
#         for i in range(lag + 1):
#             lagged_df[f'{col}_t-{i}'] = df[col].shift(i)
#     return lagged_df.dropna()

# SEQUENCE_LENGTH = 5
# cow_lag_co = lag_df(full_cow2223_co, lag=SEQUENCE_LENGTH)
# cow_lag_co.head() 

In [26]:
# # creating lag data 
# full_cow2223_ck.set_index('Date', inplace=True)

# def lag_df(df, lag=1):
#     lagged_df = pd.DataFrame()
#     for col in df.columns:
#         for i in range(lag + 1):
#             lagged_df[f'{col}_t-{i}'] = df[col].shift(i)
#     return lagged_df.dropna()

# SEQUENCE_LENGTH = 5
# cow_lag_ck = lag_df(full_cow2223_co, lag=SEQUENCE_LENGTH)
# cow_lag_ck.head() 

In [27]:
# X_train = cow_lag.drop(columns=["temperature_t-0", "relative_humidity_t-0", "wind_speed_t-0", "dew_point_temperature_t-0", "flow_value_t-0", "salmon_count_t-0"]).to_numpy().reshape(-1,6*SEQUENCE_LENGTH)
# y_train = cow_lag[["salmon_count_t-0"]].to_numpy().ravel() 

Build a Multivariant CNN 
- the following scratch is build using the framework in DSCI574- Lecture 6: Advanced Time Series Modelling 

In [28]:
# dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
#                         torch.tensor(y_train, dtype=torch.float32))
# BATCH_SIZE = 16
# dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# X, y = next(iter(dataloader))
# print(f"X shape: {X.shape}")
# print(f"y shape: {y.shape}")

In [29]:
# class CNN(nn.Module):
#     def __init__(self, input_size, output_size):
#         super().__init__()
#         self.main = nn.Sequential(
#             nn.Conv1d(input_size, 24, kernel_size=3),  
#             nn.ReLU(),
#             nn.BatchNorm1d(24),
#             nn.MaxPool1d(3),                         
#             nn.Flatten(),                            
#             nn.Linear(24, 50),                       
#             nn.ReLU(),
#             nn.Linear(50, output_size)               
#         )

#     def forward(self, x):
#         return self.main(x)

In [30]:
# model = CNN(input_size=6, output_size=6)
# optimizer = optim.Adam(model.parameters())
# criterion = nn.MSELoss()

In [31]:
# EPOCHS = 200
# for epoch in range(1, EPOCHS + 1):
#     for X_batch, y_batch in dataloader:
#         optimizer.zero_grad()
#         y_hat = model(X_batch)
#         loss = criterion(y_hat, y_batch)
#         loss.backward()
#         optimizer.step()
#     if epoch % 20 == 0:
#         print(f"Epoch {epoch}. Loss = {loss.item():.2f}")

In [32]:
# forecast_index = pd.date_range(start=full_cow2223.index[-1] + timedelta(days=14), periods=36, freq='2W')
# forecast_index

In [33]:
# def recursive_CNN_forecast(input_data, model, n=20, responses=1):
#     forecast = np.empty((n, responses))
#     for i, n in enumerate(range(n)):
#         forecast[i] = model(input_data).detach().numpy()
#         input_data = torch.cat((torch.tensor([forecast[i]], dtype=torch.float32).unsqueeze(-1),
#                                 input_data[:, :, :-1]), -1)
#     return forecast

In [34]:
# input_data = torch.tensor(X_train[-1, :], dtype=torch.float32).unsqueeze(0)
# cnn_multi = pd.DataFrame(recursive_CNN_forecast(input_data, model, n=36, responses=6),
#                          columns=["temperature", "relative_humidity", "wind_speed", "dew_point_temperature", "flow_value", "salmon_count"],
#                          index=forecast_index)


In [35]:
# df = cnn_multi.reset_index().rename(columns={'index': 'Date'})
# df['Date'] = pd.to_datetime(df['Date'])
# df_melted = df.melt("Date", var_name='Variable', value_name='Value')

# prediction_chart = alt.Chart(df_melted).mark_line().encode(
#     x='Date:T',
#     y='Value:Q',
#     color='Variable:N',
#     tooltip=['Date:T', 'Variable:N', 'Value:Q']
# ).properties(
#     width=650,
#     height=400,
#     title="Prediction") 

# prediction_chart

Linear Model 

In [36]:
# pip install statsmodels

In [37]:
# import statsmodels.api as sm

In [38]:
# X_co = cow_lag_co.drop(columns=["temperature_t-0", "relative_humidity_t-0", "wind_speed_t-0", "dew_point_temperature_t-0", "flow_value_t-0", "salmon_count_t-0"]).to_numpy().reshape(-1, 6*SEQUENCE_LENGTH)
# y_co = cow_lag_co[["salmon_count_t-0"]].to_numpy().ravel()

# X_train_co, X_test_co, y_train_co, y_test_co = train_test_split(X_co, y_co, test_size=0.5, shuffle = False )

In [39]:
# model_co = sm.GLM(
#     y_train_co,
#     X_train_co,
#     # offset=np.log(X["past.arrests"]),
#     family=sm.families.Poisson(),
# )

In [40]:
# result = model_co.fit()

# # Predicting
# predictions = result.predict(X_test_co)

In [41]:
# model_result_co = pd.DataFrame({
#     'Sample Index': range(len(predictions)),
#     'Predicted': predictions,
#     'Actual': y_test_co
# })

In [42]:
# line_pred = alt.Chart(model_result_co).mark_line(color='blue', point=True).encode(
#     x='Sample Index',
#     y='Predicted',
#     tooltip=['Sample Index', 'Predicted']
# ).properties(
#     title='Predicted vs Actual Salmon Counts'
# )

# line_actual = alt.Chart(model_result_co).mark_line(color='red', point=True).encode(
#     x='Sample Index',
#     y='Actual',
#     tooltip=['Sample Index', 'Actual']
# )

# chart = alt.layer(line_pred, line_actual).resolve_scale(
#     y = 'independent'
# )

# chart.display()

## only concern above 

In [43]:
# # X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])

# # Create the model and fit it
# model = LinearRegression()# fit_intercept=False)  # Intercept is manually added, hence fit_intercept=False
# model.fit(X_train, y_train)

In [44]:
# # X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

# # Make predictions on the test data
# y_pred = model.predict(X_test)

# # Evaluate the model performance
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# # Print the results
# print(f"Mean Squared Error: {mse}")
# print(f"R^2 Score: {r2}")

In [45]:
# r2 = model.score(X_test, y_test)
# print(r2) 

In [46]:
# X_train = sm.add_constant(X_train)
# model = sm.OLS(y_train, X_train, missing='drop')
# model_result = model.fit()

# predictions = model_result.predict(X_train)


# trained_indices = cow_lag.index 

# temp_df = pd.DataFrame(predictions, index=trained_indices, columns=['predict'])

# full_cow2223 = full_cow2223.merge(temp_df, how='left', left_index=True, right_index=True)

# print(full_cow2223.head())

In [47]:
# df_melted = full_cow2223.reset_index()
# df_melted = df_melted.melt(id_vars='Date', value_vars=['salmon_count', 'predict'], var_name='Type', value_name='Value')
# df_melted.head() 

In [48]:
# chart = alt.Chart(df_melted).mark_line(point=True).encode(
#     x='Date:T',  
#     y=alt.Y('Value:Q', title='Value'), 
#     color='Type:N',  
#     tooltip=['Date:T', 'Value:Q', 'Type:N']
# ).properties(
#     width=600,
#     height=400,
#     title='Count vs Predict Over Time'
# )

# # Display the chart
# chart.display()

Model alternative: 
- Change the count to binary then do binary logistic regression 
- Set a activation layer 

To do: 
- Skip a few days 
- Do a train test split 

Binary Logistic Regression 

In [49]:
# full_cow2223_bi = full_cow2223.copy() 
# full_cow2223_bi['salmon_present'] = np.where(full_cow2223_bi['salmon_count'] == 0, 0, 1)
# full_cow2223_bi.drop(['salmon_count'], axis=1, inplace=True)
# full_cow2223_bi.head() 

In [50]:
# chart = alt.Chart(full_cow2223_bi).mark_point().encode(
#     x='Date:T',  
#     y=alt.Y('salmon_present:Q', title='Value'), 
# ).properties(
#     width=600,
#     height=400,
#     title='Salmon Detection Over Time'
# )
# chart.show() 

In [51]:
# SEQUENCE_LENGTH = 5
# cow_bi_lag = lag_df(full_cow2223_bi, lag=SEQUENCE_LENGTH)
# cow_bi_lag.head() 

In [52]:
# X_bi = cow_bi_lag.drop(columns=["temperature_t-0", "relative_humidity_t-0", "wind_speed_t-0", "dew_point_temperature_t-0", "flow_value_t-0", "salmon_present_t-0"]).to_numpy().reshape(-1,6*SEQUENCE_LENGTH)
# y_bi = cow_bi_lag[["salmon_present_t-0"]].to_numpy().ravel() 

# X_train_bi, X_test_bi, y_train_bi, y_test_bi = train_test_split(X, y, test_size=0.5, shuffle = False )

In [53]:
from sklearn.preprocessing import StandardScaler

In [54]:
# scaler = StandardScaler()
# X_train_scaled_bi = scaler.fit_transform(X_train_bi)
# X_test_scaled_bi = scaler.transform(X_test_bi)

In [55]:
# model = LogisticRegression()
# model.fit(X_train_scaled_bi, y_train_bi)

In [56]:
# y_pred_bi = model.predict(X_test_scaled_bi)
# score_bi = model.score(X_test_scaled_bi, y_test_bi)
# print(score_bi)

In [57]:
# predictions = model.predict(X_train_scaled_bi)


# trained_indices = cow_bi_lag.index 

# temp_df = pd.DataFrame(predictions, index=trained_indices, columns=['predict'])

# full_cow2223_bi = full_cow2223_bi.merge(temp_df, how='left', left_index=True, right_index=True)

In [58]:
# df_melted_bi = full_cow2223_bi.reset_index()
# df_melted_bi = df_melted_bi.melt(id_vars='Date', value_vars=['salmon_present', 'predict'], var_name='Type', value_name='Value')
# df_melted_bi.head() 

In [59]:
# chart = alt.Chart(df_melted_bi).mark_point().encode(
#     x='Date:T',  
#     y=alt.Y('Value:Q', title='Value'), 
#     color='Type:N',  
#     tooltip=['Date:T', 'Value:Q', 'Type:N']
# ).properties(
#     width=600,
#     height=400,
#     title='Count vs Predict Over Time'
# )

# # Display the chart
# chart.display()

In [60]:
df_salmon = pd.read_csv('data/salmon_concat.csv') 
df_salmon.head() 

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co
0,2014-05-01,7,NaN,NaN,NaN,NaN,NaN,True,False
1,2014-05-02,34,NaN,NaN,NaN,NaN,NaN,True,False
2,2014-05-07,21,NaN,NaN,NaN,NaN,NaN,True,False
3,2014-05-08,136,NaN,NaN,NaN,NaN,NaN,True,False
4,2014-05-13,74,NaN,NaN,NaN,NaN,NaN,True,False


discharge of oct and nov (flow)

In [61]:
flow_cow

,STATION_NUMBER,YEAR,MONTH,DAY,FlowValue,Date
4018,08HA003,2013,1,1,10.40,2013-01-01
4019,08HA003,2013,1,2,9.18,2013-01-02
4020,08HA003,2013,1,3,8.10,2013-01-03
4021,08HA003,2013,1,4,7.69,2013-01-04
4022,08HA003,2013,1,5,7.48,2013-01-05
...,...,...,...,...,...,...
8030,08HA003,2023,12,27,15.40,2023-12-27
8031,08HA003,2023,12,28,14.10,2023-12-28
8032,08HA003,2023,12,29,15.00,2023-12-29
8033,08HA003,2023,12,30,13.00,2023-12-30


In [62]:
sorted_flow

,STATION_NUMBER,YEAR,MONTH,DAY,FlowValue,Date
0,08HA001,2013,1,1,11.50,2013-01-01
1,08HA001,2013,1,2,10.30,2013-01-02
2,08HA001,2013,1,3,9.39,2013-01-03
3,08HA001,2013,1,4,8.91,2013-01-04
4,08HA001,2013,1,5,8.65,2013-01-05
...,...,...,...,...,...,...
8030,08HA003,2023,12,27,15.40,2023-12-27
8031,08HA003,2023,12,28,14.10,2023-12-28
8032,08HA003,2023,12,29,15.00,2023-12-29
8033,08HA003,2023,12,30,13.00,2023-12-30


In [63]:
flow_cow['Date'] = pd.to_datetime(flow_cow['Date'])
flow_cow.set_index('Date', inplace=True)
flow_cow.head() 

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/1295185924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_cow['Date'] = pd.to_datetime(flow_cow['Date'])


,STATION_NUMBER,YEAR,MONTH,DAY,FlowValue
Date,,,,,
2013-01-01,08HA003,2013,1,1,10.40
2013-01-02,08HA003,2013,1,2,9.18
2013-01-03,08HA003,2013,1,3,8.10
2013-01-04,08HA003,2013,1,4,7.69
2013-01-05,08HA003,2013,1,5,7.48


In [64]:
monthly_average_flow = flow_cow.resample('M')['FlowValue'].mean()
monthly_average_flow.head() 

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/575474454.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_average_flow = flow_cow.resample('M')['FlowValue'].mean()


Date
2013-01-31    14.821290
2013-02-28    14.317143
2013-03-31    21.153548
2013-04-30     9.061333
2013-05-31     2.744194
Freq: ME, Name: FlowValue, dtype: float64

In [65]:
monthly_average_df = monthly_average_flow.to_frame(name='AverageFlowValue')
monthly_average_df.head() 

,AverageFlowValue
Date,
2013-01-31,14.821290
2013-02-28,14.317143
2013-03-31,21.153548
2013-04-30,9.061333
2013-05-31,2.744194


In [66]:
df_filtered = monthly_average_df[monthly_average_df.index.month.isin([10, 11])]
df_filtered.head() 

,AverageFlowValue
Date,
2013-10-31,4.338065
2013-11-30,6.295667
2014-10-31,7.150516
2014-11-30,16.982000
2015-10-31,2.319516


In [67]:
annual_average_flow = df_filtered.resample('A').mean()
annual_average_flow.head() 

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/1855013653.py:1: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_average_flow = df_filtered.resample('A').mean()


,AverageFlowValue
Date,
2013-12-31,5.316866
2014-12-31,12.066258
2015-12-31,11.385591
2016-12-31,22.321763
2017-12-31,19.368220


In [68]:
annual_average_flow.index = annual_average_flow.index.year  # Adjust index to show only the year
annual_average_flow.head() 

,AverageFlowValue
Date,
2013,5.316866
2014,12.066258
2015,11.385591
2016,22.321763
2017,19.368220


In [69]:
annual_average_flow.index = pd.to_datetime(annual_average_flow.index, format='%Y') + pd.DateOffset(years=1)
annual_average_flow.rename(columns={'AverageFlowValue': 'LastWinterFlow'}, inplace=True)
annual_average_flow.index = annual_average_flow.index.year  # Adjust index to show only the year

annual_average_flow.head() 

,LastWinterFlow
Date,
2014,5.316866
2015,12.066258
2016,11.385591
2017,22.321763
2018,19.368220


In [70]:
annual_average_flow.rename_axis('Year', inplace=True)
annual_average_flow.head() 

,LastWinterFlow
Year,
2014,5.316866
2015,12.066258
2016,11.385591
2017,22.321763
2018,19.368220


discharge for mar to may (flow)

In [71]:
flow35 = monthly_average_df[monthly_average_df.index.month.isin([3, 4, 5])]
flow35.head() 

,AverageFlowValue
Date,
2013-03-31,21.153548
2013-04-30,9.061333
2013-05-31,2.744194
2014-03-31,27.383871
2014-04-30,8.355000


In [72]:
flow35['Year'] = flow35.index.year
flow35['Month'] = flow35.index.month

pivot_df = flow35.pivot_table(index='Year', columns='Month', values='AverageFlowValue')
pivot_df.head() 

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/4210427450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow35['Year'] = flow35.index.year
/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/4210427450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow35['Month'] = flow35.index.month


Month,3,4,5
Year,,,
2013,21.153548,9.061333,2.744194
2014,27.383871,8.355000,3.838387
2015,11.441936,5.063667,1.485129
2016,29.127097,3.260333,0.955323
2017,27.607419,17.052667,3.889355


In [73]:
df_combined = annual_average_flow.merge(pivot_df, on='Year', how='outer')
df_combined.head() 

,LastWinterFlow,3,4,5
Year,,,,
2013,NaN,21.153548,9.061333,2.744194
2014,5.316866,27.383871,8.355000,3.838387
2015,12.066258,11.441936,5.063667,1.485129
2016,11.385591,29.127097,3.260333,0.955323
2017,22.321763,27.607419,17.052667,3.889355


In [74]:
df_combined.rename(columns={3: 'marFlow'}, inplace=True)
df_combined.rename(columns={4: 'aprFlow'}, inplace=True)
df_combined.rename(columns={5: 'mayFlow'}, inplace=True)
df_combined

,LastWinterFlow,marFlow,aprFlow,mayFlow
Year,,,,
2013,NaN,21.153548,9.061333,2.744194
2014,5.316866,27.383871,8.355000,3.838387
2015,12.066258,11.441936,5.063667,1.485129
2016,11.385591,29.127097,3.260333,0.955323
2017,22.321763,27.607419,17.052667,3.889355
2018,19.368220,5.886129,12.308667,2.240839
2019,10.152640,5.110645,5.003667,1.318065
2020,4.537952,5.619032,4.170000,2.310968
2021,14.061742,9.424194,4.140000,1.363581


temp for dec - feb

In [75]:
tem_cow

,UTC_DATE,RELATIVE_HUMIDITY,WIND_SPEED,TEMP,WINDCHILL,DEW_POINT_TEMP
0,2013-09-02,75.818182,2.727273,19.127273,NaN,14.372727
1,2013-09-03,83.125000,2.458333,18.045833,NaN,14.766667
2,2013-09-04,85.791667,2.000000,17.062500,NaN,14.379167
3,2013-09-05,94.708333,1.541667,16.837500,NaN,15.900000
4,2013-09-06,91.916667,1.583333,16.954167,NaN,15.504167
...,...,...,...,...,...,...
3918,2024-05-26,75.125000,6.791667,11.083333,NaN,6.545833
3919,2024-05-27,79.166667,4.375000,12.491667,NaN,8.745833
3920,2024-05-28,78.708333,3.291667,12.520833,NaN,8.616667
3921,2024-05-29,67.625000,4.791667,11.154167,NaN,4.970833


In [76]:
tem_cow['UTC_DATE'] = pd.to_datetime(tem_cow['UTC_DATE'])
tem_cow.set_index('UTC_DATE', inplace=True)
tem_cow.rename_axis('Date', inplace=True)
tem_cow.head() 

,RELATIVE_HUMIDITY,WIND_SPEED,TEMP,WINDCHILL,DEW_POINT_TEMP
Date,,,,,
2013-09-02,75.818182,2.727273,19.127273,NaN,14.372727
2013-09-03,83.125000,2.458333,18.045833,NaN,14.766667
2013-09-04,85.791667,2.000000,17.062500,NaN,14.379167
2013-09-05,94.708333,1.541667,16.837500,NaN,15.900000
2013-09-06,91.916667,1.583333,16.954167,NaN,15.504167


In [77]:
tem_cow

,RELATIVE_HUMIDITY,WIND_SPEED,TEMP,WINDCHILL,DEW_POINT_TEMP
Date,,,,,
2013-09-02,75.818182,2.727273,19.127273,NaN,14.372727
2013-09-03,83.125000,2.458333,18.045833,NaN,14.766667
2013-09-04,85.791667,2.000000,17.062500,NaN,14.379167
2013-09-05,94.708333,1.541667,16.837500,NaN,15.900000
2013-09-06,91.916667,1.583333,16.954167,NaN,15.504167
...,...,...,...,...,...
2024-05-26,75.125000,6.791667,11.083333,NaN,6.545833
2024-05-27,79.166667,4.375000,12.491667,NaN,8.745833
2024-05-28,78.708333,3.291667,12.520833,NaN,8.616667


In [78]:
monthly_average_temp = tem_cow.resample('M')['TEMP'].mean()
monthly_average_temp = monthly_average_temp.to_frame(name='AverageTemp')

monthly_average_temp

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/1477190486.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_average_temp = tem_cow.resample('M')['TEMP'].mean()


,AverageTemp
Date,
2013-09-30,15.603096
2013-10-31,9.480482
2013-11-30,5.516806
2013-12-31,2.316766
2014-01-31,4.547547
...,...
2024-01-31,2.931999
2024-02-29,5.328879
2024-03-31,6.326478


In [79]:
df_filtered_temp = monthly_average_temp[monthly_average_temp.index.month.isin([12, 1, 2])]
df_filtered_temp.head() 

,AverageTemp
Date,
2013-12-31,2.316766
2014-01-31,4.547547
2014-02-28,1.607777
2014-12-31,4.396715
2015-01-31,5.092313


In [80]:
df_filtered_temp

,AverageTemp
Date,
2013-12-31,2.316766
2014-01-31,4.547547
2014-02-28,1.607777
2014-12-31,4.396715
2015-01-31,5.092313
2015-02-28,7.176082
2015-12-31,4.015097
2016-01-31,3.892070
2016-02-29,6.152299


In [81]:
df_filtered_temp['RollingMean'] = df_filtered_temp['AverageTemp'].rolling(window=3, min_periods=3).mean()
df_filtered_temp 

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/2305387018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_temp['RollingMean'] = df_filtered_temp['AverageTemp'].rolling(window=3, min_periods=3).mean()


,AverageTemp,RollingMean
Date,,
2013-12-31,2.316766,NaN
2014-01-31,4.547547,NaN
2014-02-28,1.607777,2.824030
2014-12-31,4.396715,3.517346
2015-01-31,5.092313,3.698935
2015-02-28,7.176082,5.555037
2015-12-31,4.015097,5.427831
2016-01-31,3.892070,5.027750
2016-02-29,6.152299,4.686489


In [82]:
filtered_mean_temp = df_filtered_temp[df_filtered_temp.index.month.isin([2])]
filtered_mean_temp = filtered_mean_temp.drop('AverageTemp', axis=1)
filtered_mean_temp 

,RollingMean
Date,
2014-02-28,2.824030
2015-02-28,5.555037
2016-02-29,4.686489
2017-02-28,1.328632
2018-02-28,3.191699
2019-02-28,2.796946
2020-02-29,4.168349
2021-02-28,3.905157
2022-02-28,2.567577


In [83]:
filtered_mean_temp.index = filtered_mean_temp.index.year  # Adjust index to show only the year
filtered_mean_temp.rename_axis('Year', inplace=True)
filtered_mean_temp.rename(columns={'RollingMean': 'LastWinterTemp'}, inplace=True)

filtered_mean_temp.head() 

,LastWinterTemp
Year,
2014,2.824030
2015,5.555037
2016,4.686489
2017,1.328632
2018,3.191699


temp for mar - may 

In [84]:
df_filtered_marmay = monthly_average_temp[monthly_average_temp.index.month.isin([3,4,5])]
df_filtered_marmay

,AverageTemp
Date,
2014-03-31,5.937401
2014-04-30,9.200833
2014-05-31,13.870775
2015-03-31,7.991129
2015-04-30,8.587107
2015-05-31,14.717418
2016-03-31,7.512500
2016-04-30,11.871944
2016-05-31,14.515945


In [85]:
df_filtered_marmay['Year'] = df_filtered_marmay.index.year
df_filtered_marmay['Month'] = df_filtered_marmay.index.month

pivot_df_temp = df_filtered_marmay.pivot_table(index='Year', columns='Month', values='AverageTemp')
pivot_df_temp.head() 

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/4141215061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_marmay['Year'] = df_filtered_marmay.index.year
/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/4141215061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_marmay['Month'] = df_filtered_marmay.index.month


Month,3,4,5
Year,,,
2014,5.937401,9.200833,13.870775
2015,7.991129,8.587107,14.717418
2016,7.512500,11.871944,14.515945
2017,5.671237,8.566111,13.086828
2018,5.477688,9.018712,15.219624


In [86]:
df_combined_temp = filtered_mean_temp.merge(pivot_df_temp, on='Year', how='outer')
df_combined_temp.head() 

,LastWinterTemp,3,4,5
Year,,,,
2014,2.824030,5.937401,9.200833,13.870775
2015,5.555037,7.991129,8.587107,14.717418
2016,4.686489,7.512500,11.871944,14.515945
2017,1.328632,5.671237,8.566111,13.086828
2018,3.191699,5.477688,9.018712,15.219624


In [87]:
df_combined_temp.rename(columns={3: 'marTemp'}, inplace=True)
df_combined_temp.rename(columns={4: 'aprTemp'}, inplace=True)
df_combined_temp.rename(columns={5: 'mayTemp'}, inplace=True)
df_combined_temp.head() 

,LastWinterTemp,marTemp,aprTemp,mayTemp
Year,,,,
2014,2.824030,5.937401,9.200833,13.870775
2015,5.555037,7.991129,8.587107,14.717418
2016,4.686489,7.512500,11.871944,14.515945
2017,1.328632,5.671237,8.566111,13.086828
2018,3.191699,5.477688,9.018712,15.219624


In [88]:
flowTemp = df_combined_temp.merge(df_combined, on='Year', how='outer')
flowTemp.head() 

,LastWinterTemp,marTemp,aprTemp,mayTemp,LastWinterFlow,marFlow,aprFlow,mayFlow
Year,,,,,,,,
2013,NaN,NaN,NaN,NaN,NaN,21.153548,9.061333,2.744194
2014,2.824030,5.937401,9.200833,13.870775,5.316866,27.383871,8.355000,3.838387
2015,5.555037,7.991129,8.587107,14.717418,12.066258,11.441936,5.063667,1.485129
2016,4.686489,7.512500,11.871944,14.515945,11.385591,29.127097,3.260333,0.955323
2017,1.328632,5.671237,8.566111,13.086828,22.321763,27.607419,17.052667,3.889355


In [89]:
# df_salmon.set_index('date', inplace=True)
# cols = [col for col in df_salmon.columns if col != 'count'] + ['count']
# df_salmon = df_salmon[cols]
# df_salmon.index = pd.to_datetime(df_salmon.index)

df_salmon.head() 

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co
0,2014-05-01,7,NaN,NaN,NaN,NaN,NaN,True,False
1,2014-05-02,34,NaN,NaN,NaN,NaN,NaN,True,False
2,2014-05-07,21,NaN,NaN,NaN,NaN,NaN,True,False
3,2014-05-08,136,NaN,NaN,NaN,NaN,NaN,True,False
4,2014-05-13,74,NaN,NaN,NaN,NaN,NaN,True,False


In [90]:
df_salmon['date'] = pd.to_datetime(df_salmon['date'])  # Convert 'date' to datetime

# Create a mask for April 15 to October 15 for any year
mask = ((df_salmon['date'].dt.month > 4) | ((df_salmon['date'].dt.month == 4) & (df_salmon['date'].dt.day >= 15))) & \
       ((df_salmon['date'].dt.month < 10) | ((df_salmon['date'].dt.month == 10) & (df_salmon['date'].dt.day <= 15)))

# Apply mask to filter data
df_filtered = df_salmon.loc[mask]

# Determine the range of years in your data
start_year = df_filtered['date'].dt.year.min()
end_year = df_filtered['date'].dt.year.max()

# Generate all dates from April 15 to October 15 for each year within the range
all_dates = []
for year in range(start_year, end_year + 1):
    start_date = f"{year}-04-15"
    end_date = f"{year}-10-15"
    all_dates.extend(pd.date_range(start=start_date, end=end_date))

# Check which dates are missing
existing_dates = set(df_filtered['date'])
missing_dates = [date for date in all_dates if date not in existing_dates]

# Create a DataFrame for missing dates with all other columns filled with 0 or appropriate defaults
df_missing = pd.DataFrame({
    'date': missing_dates,
    'count': [0] * len(missing_dates),
    '70.2': [0] * len(missing_dates),
    'cow bay': [0] * len(missing_dates),
    'mainstem fence': [0] * len(missing_dates),
    'skutz': [0] * len(missing_dates),
    'vimy pool': [0] * len(missing_dates),
    'ck': [False] * len(missing_dates),
    'co': [False] * len(missing_dates)
})

# Concatenate the original filtered data with the missing data DataFrame
df_final = pd.concat([df_filtered, df_missing]).sort_values('date')

In [91]:
df_final

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co
0,2014-04-15,0,0,0,0,0,0,False,False
1,2014-04-16,0,0,0,0,0,0,False,False
2,2014-04-17,0,0,0,0,0,0,False,False
3,2014-04-18,0,0,0,0,0,0,False,False
4,2014-04-19,0,0,0,0,0,0,False,False
...,...,...,...,...,...,...,...,...,...
1423,2023-10-11,0,0,0,0,0,0,False,False
1424,2023-10-12,0,0,0,0,0,0,False,False
1425,2023-10-13,0,0,0,0,0,0,False,False
1426,2023-10-14,0,0,0,0,0,0,False,False


In [92]:
df_final.replace({True: 1, False: 0, np.nan: 0}, inplace=True)
df_final.fillna(0, inplace=True)
df_final.head()

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/1657996409.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final.replace({True: 1, False: 0, np.nan: 0}, inplace=True)


,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co
0,2014-04-15,0,0.0,0.0,0.0,0.0,0.0,0,0
1,2014-04-16,0,0.0,0.0,0.0,0.0,0.0,0,0
2,2014-04-17,0,0.0,0.0,0.0,0.0,0.0,0,0
3,2014-04-18,0,0.0,0.0,0.0,0.0,0.0,0,0
4,2014-04-19,0,0.0,0.0,0.0,0.0,0.0,0,0


In [93]:
df_final['Year'] = df_final['date'].dt.year
df_final.head() 

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,Year
0,2014-04-15,0,0.0,0.0,0.0,0.0,0.0,0,0,2014
1,2014-04-16,0,0.0,0.0,0.0,0.0,0.0,0,0,2014
2,2014-04-17,0,0.0,0.0,0.0,0.0,0.0,0,0,2014
3,2014-04-18,0,0.0,0.0,0.0,0.0,0.0,0,0,2014
4,2014-04-19,0,0.0,0.0,0.0,0.0,0.0,0,0,2014


In [94]:
wMacro = df_final.merge(flowTemp, how='left', left_on='Year', right_index=True)
wMacro.head() 

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,Year,LastWinterTemp,marTemp,aprTemp,mayTemp,LastWinterFlow,marFlow,aprFlow,mayFlow
0,2014-04-15,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387
1,2014-04-16,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387
2,2014-04-17,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387
3,2014-04-18,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387
4,2014-04-19,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387


In [95]:
tem_cow

,RELATIVE_HUMIDITY,WIND_SPEED,TEMP,WINDCHILL,DEW_POINT_TEMP
Date,,,,,
2013-09-02,75.818182,2.727273,19.127273,NaN,14.372727
2013-09-03,83.125000,2.458333,18.045833,NaN,14.766667
2013-09-04,85.791667,2.000000,17.062500,NaN,14.379167
2013-09-05,94.708333,1.541667,16.837500,NaN,15.900000
2013-09-06,91.916667,1.583333,16.954167,NaN,15.504167
...,...,...,...,...,...
2024-05-26,75.125000,6.791667,11.083333,NaN,6.545833
2024-05-27,79.166667,4.375000,12.491667,NaN,8.745833
2024-05-28,78.708333,3.291667,12.520833,NaN,8.616667


In [96]:
df_joined = pd.merge(wMacro, tem_cow, how='left', left_on='date', right_index = True)
df_joined.head() 

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,Year,...,mayTemp,LastWinterFlow,marFlow,aprFlow,mayFlow,RELATIVE_HUMIDITY,WIND_SPEED,TEMP,WINDCHILL,DEW_POINT_TEMP
0,2014-04-15,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,13.870775,5.316866,27.383871,8.355,3.838387,52.416667,10.375000,11.637500,NaN,1.929167
1,2014-04-16,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,13.870775,5.316866,27.383871,8.355,3.838387,88.291667,4.333333,9.508333,NaN,7.583333
2,2014-04-17,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,13.870775,5.316866,27.383871,8.355,3.838387,88.208333,6.041667,9.975000,NaN,8.050000
3,2014-04-18,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,13.870775,5.316866,27.383871,8.355,3.838387,63.291667,8.041667,9.441667,NaN,2.550000
4,2014-04-19,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,13.870775,5.316866,27.383871,8.355,3.838387,75.250000,3.250000,8.529167,NaN,4.150000


In [97]:
df_joined.drop(['RELATIVE_HUMIDITY', 'WIND_SPEED', 'WINDCHILL', 'DEW_POINT_TEMP'], axis=1, inplace=True)
df_joined.head() 

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,Year,LastWinterTemp,marTemp,aprTemp,mayTemp,LastWinterFlow,marFlow,aprFlow,mayFlow,TEMP
0,2014-04-15,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387,11.637500
1,2014-04-16,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387,9.508333
2,2014-04-17,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387,9.975000
3,2014-04-18,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387,9.441667
4,2014-04-19,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,2.82403,5.937401,9.200833,13.870775,5.316866,27.383871,8.355,3.838387,8.529167


In [98]:
flow_cow

,STATION_NUMBER,YEAR,MONTH,DAY,FlowValue
Date,,,,,
2013-01-01,08HA003,2013,1,1,10.40
2013-01-02,08HA003,2013,1,2,9.18
2013-01-03,08HA003,2013,1,3,8.10
2013-01-04,08HA003,2013,1,4,7.69
2013-01-05,08HA003,2013,1,5,7.48
...,...,...,...,...,...
2023-12-27,08HA003,2023,12,27,15.40
2023-12-28,08HA003,2023,12,28,14.10
2023-12-29,08HA003,2023,12,29,15.00


In [99]:
df_complete = pd.merge(df_joined, flow_cow, how='left', left_on='date', right_index = True)
df_complete

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,Year,...,LastWinterFlow,marFlow,aprFlow,mayFlow,TEMP,STATION_NUMBER,YEAR,MONTH,DAY,FlowValue
0,2014-04-15,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,5.316866,27.383871,8.355000,3.838387,11.637500,08HA003,2014,4,15,4.300
1,2014-04-16,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,5.316866,27.383871,8.355000,3.838387,9.508333,08HA003,2014,4,16,4.570
2,2014-04-17,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,5.316866,27.383871,8.355000,3.838387,9.975000,08HA003,2014,4,17,6.020
3,2014-04-18,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,5.316866,27.383871,8.355000,3.838387,9.441667,08HA003,2014,4,18,7.470
4,2014-04-19,0,0.0,0.0,0.0,0.0,0.0,0,0,2014,...,5.316866,27.383871,8.355000,3.838387,8.529167,08HA003,2014,4,19,6.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,2023-10-11,0,0.0,0.0,0.0,0.0,0.0,0,0,2023,...,1.746417,7.648065,12.996333,2.918065,12.458333,08HA003,2023,10,11,0.571
1424,2023-10-12,0,0.0,0.0,0.0,0.0,0.0,0,0,2023,...,1.746417,7.648065,12.996333,2.918065,9.975000,08HA003,2023,10,12,0.597
1425,2023-10-13,0,0.0,0.0,0.0,0.0,0.0,0,0,2023,...,1.746417,7.648065,12.996333,2.918065,10.087500,08HA003,2023,10,13,0.524
1426,2023-10-14,0,0.0,0.0,0.0,0.0,0.0,0,0,2023,...,1.746417,7.648065,12.996333,2.918065,12.341667,08HA003,2023,10,14,0.496


In [100]:
df_complete.drop(['STATION_NUMBER', 'YEAR', 'DAY', 'Year'], axis=1, inplace=True)
df_complete

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,LastWinterTemp,marTemp,aprTemp,mayTemp,LastWinterFlow,marFlow,aprFlow,mayFlow,TEMP,MONTH,FlowValue
0,2014-04-15,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,9.200833,13.870775,5.316866,27.383871,8.355000,3.838387,11.637500,4,4.300
1,2014-04-16,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,9.200833,13.870775,5.316866,27.383871,8.355000,3.838387,9.508333,4,4.570
2,2014-04-17,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,9.200833,13.870775,5.316866,27.383871,8.355000,3.838387,9.975000,4,6.020
3,2014-04-18,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,9.200833,13.870775,5.316866,27.383871,8.355000,3.838387,9.441667,4,7.470
4,2014-04-19,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,9.200833,13.870775,5.316866,27.383871,8.355000,3.838387,8.529167,4,6.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,2023-10-11,0,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,5.093952,7.577862,15.933737,1.746417,7.648065,12.996333,2.918065,12.458333,10,0.571
1424,2023-10-12,0,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,5.093952,7.577862,15.933737,1.746417,7.648065,12.996333,2.918065,9.975000,10,0.597
1425,2023-10-13,0,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,5.093952,7.577862,15.933737,1.746417,7.648065,12.996333,2.918065,10.087500,10,0.524
1426,2023-10-14,0,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,5.093952,7.577862,15.933737,1.746417,7.648065,12.996333,2.918065,12.341667,10,0.496


In [101]:
def lag_df(df, date_col, columns_to_lag, lag=1):
    # Make sure the DataFrame is sorted by date
    df = df.sort_values(date_col)
    
    # Initialize an empty DataFrame to store the lagged data
    lagged_df = pd.DataFrame(index=df.index)
    
    # Loop through each specified column in the DataFrame
    for col in columns_to_lag:
        # Create lagged versions of the column
        for i in range(1, lag + 1):
            lagged_name = f'{col}_t-{i}'  # Create a name for the lagged column
            lagged_df[lagged_name] = df[col].shift(i)
    
    # Combine the original DataFrame with the lagged DataFrame
    result_df = pd.concat([df, lagged_df], axis=1)
    
    result_df.dropna(inplace=True)

    return result_df

In [102]:
columns_to_lag = ['TEMP', 'FlowValue']

lagged_data = lag_df(df_complete, 'date', columns_to_lag, lag=5)
lagged_data

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,LastWinterTemp,...,TEMP_t-1,TEMP_t-2,TEMP_t-3,TEMP_t-4,TEMP_t-5,FlowValue_t-1,FlowValue_t-2,FlowValue_t-3,FlowValue_t-4,FlowValue_t-5
5,2014-04-20,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,...,8.529167,9.441667,9.975000,9.508333,11.637500,6.110,7.470,6.020,4.570,4.300
6,2014-04-21,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,...,7.950000,8.529167,9.441667,9.975000,9.508333,6.980,6.110,7.470,6.020,4.570
7,2014-04-22,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,...,8.912500,7.950000,8.529167,9.441667,9.975000,6.970,6.980,6.110,7.470,6.020
8,2014-04-23,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,...,8.987500,8.912500,7.950000,8.529167,9.441667,6.990,6.970,6.980,6.110,7.470
9,2014-04-24,0,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,...,7.875000,8.987500,8.912500,7.950000,8.529167,6.500,6.990,6.970,6.980,6.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,2023-10-11,0,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,...,11.633333,14.037500,14.262500,14.737500,13.529167,0.418,0.316,0.315,0.320,0.331
1424,2023-10-12,0,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,...,12.458333,11.633333,14.037500,14.262500,14.737500,0.571,0.418,0.316,0.315,0.320
1425,2023-10-13,0,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,...,9.975000,12.458333,11.633333,14.037500,14.262500,0.597,0.571,0.418,0.316,0.315
1426,2023-10-14,0,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,...,10.087500,9.975000,12.458333,11.633333,14.037500,0.524,0.597,0.571,0.418,0.316


In [103]:
cols = [col for col in lagged_data.columns if col != 'count'] + ['count']
lagged_data = lagged_data[cols]

In [104]:
from sklearn.linear_model import Ridge  
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split


In [106]:
X = lagged_data.drop(columns=["count", 'date'])
y = lagged_data["count"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
) 

In [107]:
pipe = make_pipeline(StandardScaler(), Ridge())

In [108]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('ridge', Ridge())])

In [109]:
kk = pipe.predict(X_test)

In [110]:
predict = pd.DataFrame(kk, columns = ['prediction']) 

In [111]:
test = pd.DataFrame(y_test).reset_index() 

In [112]:
test.head() 

,index,count
0,443,1
1,699,0
2,578,26
3,189,0
4,248,0


In [113]:
predict

,prediction
0,297.488212
1,38.074959
2,183.078812
3,-27.135240
4,72.133795
...,...
385,-14.238300
386,205.229091
387,79.790842
388,-2.588969


In [114]:
test

,index,count
0,443,1
1,699,0
2,578,26
3,189,0
4,248,0
...,...,...
385,900,0
386,144,661
387,252,12
388,1373,0


In [115]:
df_horizontal = pd.concat([test, predict], axis=1)
df_horizontal.head() 

,index,count,prediction
0,443,1,297.488212
1,699,0,38.074959
2,578,26,183.078812
3,189,0,-27.135240
4,248,0,72.133795


In [116]:
df_horizontal['test_bi'] = df_horizontal['count'].apply(lambda x: 1 if x >= 1 else 0)
df_horizontal.head() 

,index,count,prediction,test_bi
0,443,1,297.488212,1
1,699,0,38.074959,0
2,578,26,183.078812,1
3,189,0,-27.135240,0
4,248,0,72.133795,0


In [117]:
df_sorted = df_horizontal.sort_values('prediction')
df_sorted.drop(['index'], axis=1, inplace=True)
df_sorted.head() 

,count,prediction,test_bi
186,0,-481.461468,0
283,0,-335.122044,0
274,0,-72.881373,0
226,0,-59.840298,0
191,0,-57.849194,0


In [118]:
chart = alt.Chart(df_sorted).mark_point().encode(
    x='index:Q',  # Quantitative scale for indices
    y= 'prediction:Q',
    color='test_bi:N'  # Nominal scale for binary values, differentiating lines by color
).transform_window(
    index='count()'  # Generate an index for x-axis
)

In [119]:
chart 

alt.Chart(...)

We are interested in whether or not tagging should happen ionstead of the exact count value, so maybe another scoring method will be more suitable. 

Logistic Regression 

In [120]:
lagged_data

,date,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,LastWinterTemp,marTemp,...,TEMP_t-2,TEMP_t-3,TEMP_t-4,TEMP_t-5,FlowValue_t-1,FlowValue_t-2,FlowValue_t-3,FlowValue_t-4,FlowValue_t-5,count
5,2014-04-20,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,...,9.441667,9.975000,9.508333,11.637500,6.110,7.470,6.020,4.570,4.300,0
6,2014-04-21,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,...,8.529167,9.441667,9.975000,9.508333,6.980,6.110,7.470,6.020,4.570,0
7,2014-04-22,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,...,7.950000,8.529167,9.441667,9.975000,6.970,6.980,6.110,7.470,6.020,0
8,2014-04-23,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,...,8.912500,7.950000,8.529167,9.441667,6.990,6.970,6.980,6.110,7.470,0
9,2014-04-24,0.0,0.0,0.0,0.0,0.0,0,0,2.824030,5.937401,...,8.987500,8.912500,7.950000,8.529167,6.500,6.990,6.970,6.980,6.110,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,2023-10-11,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,5.093952,...,14.037500,14.262500,14.737500,13.529167,0.418,0.316,0.315,0.320,0.331,0
1424,2023-10-12,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,5.093952,...,11.633333,14.037500,14.262500,14.737500,0.571,0.418,0.316,0.315,0.320,0
1425,2023-10-13,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,5.093952,...,12.458333,11.633333,14.037500,14.262500,0.597,0.571,0.418,0.316,0.315,0
1426,2023-10-14,0.0,0.0,0.0,0.0,0.0,0,0,3.049505,5.093952,...,9.975000,12.458333,11.633333,14.037500,0.524,0.597,0.571,0.418,0.316,0


In [121]:
lagged_data['count_bi'] = lagged_data['count'].apply(lambda x: 1 if x >= 1 else 0)
lagged_data.head() 

/var/folders/tf/pzsl6f5j1wlcsbq4m08zvbtw0000gn/T/ipykernel_39655/2230237863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_data['count_bi'] = lagged_data['count'].apply(lambda x: 1 if x >= 1 else 0)


,date,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,LastWinterTemp,marTemp,...,TEMP_t-3,TEMP_t-4,TEMP_t-5,FlowValue_t-1,FlowValue_t-2,FlowValue_t-3,FlowValue_t-4,FlowValue_t-5,count,count_bi
5,2014-04-20,0.0,0.0,0.0,0.0,0.0,0,0,2.82403,5.937401,...,9.975000,9.508333,11.637500,6.11,7.47,6.02,4.57,4.30,0,0
6,2014-04-21,0.0,0.0,0.0,0.0,0.0,0,0,2.82403,5.937401,...,9.441667,9.975000,9.508333,6.98,6.11,7.47,6.02,4.57,0,0
7,2014-04-22,0.0,0.0,0.0,0.0,0.0,0,0,2.82403,5.937401,...,8.529167,9.441667,9.975000,6.97,6.98,6.11,7.47,6.02,0,0
8,2014-04-23,0.0,0.0,0.0,0.0,0.0,0,0,2.82403,5.937401,...,7.950000,8.529167,9.441667,6.99,6.97,6.98,6.11,7.47,0,0
9,2014-04-24,0.0,0.0,0.0,0.0,0.0,0,0,2.82403,5.937401,...,8.912500,7.950000,8.529167,6.50,6.99,6.97,6.98,6.11,0,0


In [126]:
X_1 = lagged_data.drop(columns=["count", 'date', 'count_bi'])
y_1 = lagged_data["count_bi"]

X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(
    X_1, y_1, test_size=0.2, random_state=123
) 

In [127]:
pipe_1 = make_pipeline(StandardScaler(), LogisticRegression())

In [128]:
y_1_test

443     1
699     0
578     1
189     0
248     0
       ..
900     0
144     1
252     1
1373    0
60      0
Name: count_bi, Length: 390, dtype: int64

In [131]:
kk_1 = pipe_1.predict(X_1_test)
kk_1

array([1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,

In [132]:
pipe_1.fit(X_1_train, y_1_train)
pipe_1.score(X_1_test, y_1_test)

1.0